In [1]:
#CSC620
#HA#5
#Paula Abigail Tam <921850992>

#first 4 imports have their documentation here: https://www.nltk.org/_modules/nltk/util.html
#found documentation on nltk.lm package as a whole at: https://www.nltk.org/api/nltk.lm.html
#other documentation on nltk.lm.preprocessing specifically is at: https://www.nltk.org/api/nltk.lm.preprocessing.html

from nltk.util import pad_sequence #pads a sequence so there are spaces in between each item
    #the parameters are the initial sequence, the ngram degree, boolean if there is a pad to the left or right, and an option symbol to add to the left or right pad
from nltk.util import bigrams #This import is what I used for HA#4. This is for bigrams specifically.
from nltk.util import ngrams #This import is for any kind of ngram, you can indicate what kind of ngram with the second parameter.
from nltk.util import everygrams #This import returns all of the possible ngrams.
    #e.g. the string 'a b c' will give [(a), (a, b), (a, b, c), (b), (b, c), (c)]
from nltk.lm.preprocessing import pad_both_ends #does the same thing as pad_sequence with all the parameters filled
    #more convenient to use pad_both_ends if you do not want to write out all the parameters for pad_sequence
from nltk.lm.preprocessing import flatten #makes a list of ngrams into a flat list

text = [['I','need','to','book', 'ticket', 'to', 'Australia' ], ['I', 'want', 'to' ,'read', 'a' ,'book', 'of' ,'Shakespeare']]
    #text is a tuple with a sentence (as a list of strings) in each element

list(bigrams(text[0])) #so text[0] gets the first sentence: ['I','need','to','book', 'ticket', 'to', 'Australia']
#print(list(bigrams(text[0])) #gives us:
    #[('I', 'need'), ('need', 'to'), ('to', 'book'), ('book', 'ticket'), ('ticket', 'to'), ('to', 'Australia')]

list(ngrams(text[1], n=3)) #The n=3 makes it so that the ngrams that are given are trigrams.
    #text[1] gives us the second sentence: ['I', 'want', 'to' ,'read', 'a' ,'book', 'of' ,'Shakespeare']
    #so all together, this gives us a list of the trigrams of the second sentence.
#print(list(ngrams(text[1], n=3))) #gives us:
    #[('I', 'want', 'to'), ('want', 'to', 'read'), ('to', 'read', 'a'), ('read', 'a', 'book'), ('a', 'book', 'of'), ('book', 'of', 'Shakespeare')]
    
#Runnning the cell will just show the list of trigrams, as it is last

[('I', 'want', 'to'),
 ('want', 'to', 'read'),
 ('to', 'read', 'a'),
 ('read', 'a', 'book'),
 ('a', 'book', 'of'),
 ('book', 'of', 'Shakespeare')]

In [ ]:
import pandas as pd #importing the pandas dataframe
df = pd.read_csv('./trump-tweets/realdonaldtrump.csv') #data frame is kinda like a data structure, which means we can treat it like an object
    #edited relative path from '../input/trump-tweets/realdonaldtrump.csv' to './trump-tweets/realdonaldtrump.csv' in order to correctly access the dataset
df.head() #returns the first n rows (default n = 5) for the object based on position
    #the pandas.DataFrame.head documentation can be found at: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html

from nltk import word_tokenize, sent_tokenize #for word and sentence tokenization
trump_corpus = list(df['content'].apply(word_tokenize)) #the tweets are in 'content' and .apply(word_tokenize) applies the word tokenizer to all the tweets
    #trump_corpus is a list of all the tokenized tweets
    
print(trump_corpus)

In [2]:
#Separate cell since it is the second example in the blog post

#I had to "pip install opendatasets" in order to obtain the trump-tweets dataset from Kaggle that the blog was using
#I also had to make a Kaggle account in order to download the dataset
#I followed this blog to get the dataset into jupyter notebook: https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/
#import opendatasets as od
#od.download("https://www.kaggle.com/datasets/austinreese/trump-tweets") #based on the example in the blog we are following, this is the "trump-tweets" dataset they used

import pandas as pd #importing the pandas dataframe
df = pd.read_csv('./trump-tweets/realdonaldtrump.csv') #data frame is kinda like a data structure, which means we can treat it like an object
    #edited relative path from '../input/trump-tweets/realdonaldtrump.csv' to './trump-tweets/realdonaldtrump.csv' in order to correctly access the dataset
df.head() #returns the first n rows (default n = 5) for the object based on position
    #the pandas.DataFrame.head documentation can be found at: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html

from nltk import word_tokenize, sent_tokenize #for word and sentence tokenization
trump_corpus = list(df['content'].apply(word_tokenize)) #the tweets are in 'content' and .apply(word_tokenize) applies the word tokenizer to all the tweets
    #trump_corpus is a list of all the tokenized tweets

from nltk.lm.preprocessing import padded_everygram_pipeline #does both everygram and flattens in that order
# Preprocess the tokenized text for 3-grams language modelling
n = 3 #They had this variable n=3 for trigrams. If we wanted to use 4-grams instead, we would only have to change this one variable from 3 to 4
    #hooray for reusability!
train_data, padded_sents = padded_everygram_pipeline(n, trump_corpus) #train_data and padded_sents are lazy iterators
    #so we are padding and flattening the trump_corpus in trigrams (because n=3)

from nltk.lm import MLE #MLE stands for Maximum Likelihood Estimator
trump_model = MLE(n) # Lets train a 3-grams model, previously we set n=3
trump_model.fit(train_data, padded_sents) #.fit trains the model to the text/data given 
    #since both train_data (the data) and padded_sents (the vocab) are both arguments, this is a supervised model

#documentation on the nltk.tokenize.treebank is found at: https://www.nltk.org/_modules/nltk/tokenize/treebank.html
#accourding to the documentation, (copy pasted from the page ^), the how it tokenizes is
    #- split standard contractions, e.g. ``don't`` -> ``do n't`` and ``they'll`` -> ``they 'll``
    #- treat most punctuation characters as separate tokens
    #- split off commas and single quotes, when followed by whitespace
    #- separate periods that appear at the end of line
from nltk.tokenize.treebank import TreebankWordDetokenizer #the detokenizer uses reverse regex to remove padding, combine contractions, and does not keep the original whitespaces
    #e.g. ["I went\n to the\n store."] will detokenize to "I went to the store." (if I understood the documentation correctly)
    #e.g. the tokenized list ["I", "did", "n't", "know", "."] will detokenize to "I didn't know."
detokenize = TreebankWordDetokenizer().detokenize #another use of reusability, so you dont have to type TreebankWordDetokenizer().detokenize whenever you wanna detokenize

def generate_sent(model, num_words, random_seed=42): #parameters are the model we trained, the number of words to print out, and which seed (pick from 0-42)
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = [] #empty list to store the content
    for token in model.generate(num_words, random_seed=random_seed): #loop through the generated string that was made based on the trained model
            #.generate() generates a string of text based off the trigram models that we trained the model in
            #if we did not have the random_seed, then the sentence generated would (most likely) be different everytime
        if token == '<s>': #if start of sentence, continue to the next word
            continue
        if token == '</s>': #if end of sentence, break for loop
            break
        content.append(token) #append the string into the existing string in the list
    return detokenize(content) #return the detokenized version of the list of content

generate_sent(trump_model, num_words=10, random_seed=5) #call the function generate_sent

KeyboardInterrupt: 